# Edge AI Anomaly Detection

---

## Model building and training

### Overview

We will now use the data collected in the previous section to train an artificial neural network-based Anomaly Detector of our design. To this end we will use an Autoencoder model. To understand why we choose such model we need to understand how it works. From [Wikipedia](https://en.wikipedia.org/wiki/Autoencoder):

> An autoencoder is a type of artificial neural network used to learn efficient codings of unlabeled data (unsupervised learning). The encoding is validated and refined by attempting to regenerate the input from the encoding. The autoencoder learns a representation (encoding) for a set of data, typically for dimensionality reduction, by training the network to ignore insignificant data (“noise”).

> Another application for autoencoders is **anomaly detection**. By learning to replicate the most salient features in the training data [...] the model is encouraged to learn to precisely reproduce the most frequently observed characteristics. When facing anomalies, the model should worsen its reconstruction performance. In most cases, only data with normal instances are used to train the autoencoder; in others, the frequency of anomalies is small compared to the observation set so that its contribution to the learned representation could be ignored. **After training, the autoencoder will accurately reconstruct "normal" data, while failing to do so with unfamiliar anomalous data**. Reconstruction error (the error between the original data and its low dimensional reconstruction) is used as an anomaly score to detect anomalies

In simple terms: 
- The Autoencoder is a artificial neural network model that learns how to reconstruct the input data at the output. 
- If trained on "normal" data, it learns to recontruct **only normal data** and fails to reconstruct anomalies.
- We can detect anomalies by computing the reconstruction error of the Autoencoder. If the error is above a certain threshold (which we will decide) the input sample is an anomaly.

Why did we choose this approach over others?
- The Autoencoder falls in the "[Unsupervised Learning](https://en.wikipedia.org/wiki/Unsupervised_learning)" category: it doesn't need labeled data to be trained i.e. we don't need to go through all the dataset and manually label the samples as "normal" or "anomaly" ([Supervised Learning](https://en.wikipedia.org/wiki/Supervised_learning)).
- Simpler data collection: we just need to provide it with the "normal" data. We don't need to artificially generate anomalies to train it on them.

### Data Processing

We can now work on our `train-data-raw.csv`. The dataset is provided within this repository.

If you're running this notebook through Google Colab you'll need to download the dataset running the cell below:

In [ ]:
!ls *.csv

Let's start taking a look at the content of this dataset, we'll use [pandas](https://pandas.pydata.org/) (Python Data Analysis library) for this.

In [ ]:
import pandas as pd

raw_data = pd.read_csv("./train-data-raw.csv")

raw_data.head()

#### Feature selection

As you might notice there's some information in the dataset we don't care about and are not meaningful for our application:
- `ID`
- The various `timestamps`
- `assetName` which doesn't change

Then we can remove them from the dataset.

In [ ]:
features = ['ACC_Y', 'ACC_X', 'ACC_Z',
            'PRESSURE', 'TEMP_PRESS', 'TEMP_HUM',
            'HUMIDITY', 'GYRO_X', 'GYRO_Y', 'GYRO_Z']

data = raw_data[features]

data.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

data.hist(bins=50, figsize=(20,15))
plt.show()

**Note**: Some of you might notice that this is a really simple dataset: some of the input data (like `GYRO_*` and `ACC_*`) do not change much over time. Such a dataset is not very challenging and a few, well-placed, thresholds might be sufficient to spot anomalous behaviour. For this tutorial we decided to keep things simple and easy to replicate. Anomalies can be simply triggered by moving the Raspberry Pi around.

Keep in mind that this approach is generic: any dataset from any appliance/connected device can be processed in the same way we're showing here. That's the magic of neural networks!

#### Feature scaling

AI models don't perform well when the input numerical attributes have very different scales. As you can see `ACC_X`, `ACC_Y` and `ACC_Z` range from 0 to 1, while the `PRESSURE` have far higher values.

There are two common ways to address this: _normalization_ and _standardization_.

_Normalization_ (a.k.a. Min-max scaling) shifts and rescales values so that they end up ranging from 0 to 1. This can be done by subtracting the min value and dividing by the max minus the min.

x' = $\frac{x - min(x)}{max(x) - min(x)}$

_Standardization_ makes the values of each feature in the data have zero-mean (when subtracting the mean in the numerator) and unit-variance. The general method of calculation is to determine the distribution mean and standard deviation for each feature. Next we subtract the mean from each feature. Then we divide the values (mean is already subtracted) of each feature by its standard deviation.

x' = $\frac{x - avg(x)}{\sigma}$

Fortunately for us [scikit-learn](https://scikit-learn.org/stable/) library provides a function for both of them. In this case we'll use _normalization_ because it works well for this application.

In [ ]:
print("Data used in the Triton preprocessor")
print("-----------Min-----------")
print(data.min())
print("-----------Max-----------")
print(data.max())
print("-------------------------")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.to_numpy())

In [ ]:
pd.DataFrame(scaled_data).describe()

#### Train test split

The only way to know how well a model will generalize to new data points is to try it on new data. To do so we split our data into two sets: the training set and the test set.

To do so we'll use a function from `scikit-learn`.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_test = train_test_split(scaled_data, test_size=0.3, random_state=42)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

---

### Model training

We can now leverage the [Keras](https://keras.io/) API of [Tensorflow](https://www.tensorflow.org/) for creating our Autoencoder and then train it on our dataset.

We'll design a neural network architecture such that we impose a bottleneck in the network which forces a compressed knowledge representation of the original input (also called the _latent-space representation_). If the input features were each independent of one another, this compression and subsequent reconstruction would be a very difficult task. However, if some sort of structure exists in the data (ie. correlations between input features), this structure can be learned and consequently leveraged when forcing the input through the network's bottleneck.

The bottleneck consists of reducing the number of neurons for each layer of the neural network up to a certain point, and then increase the number until the original input number is reached. This will result in a hourglass shape which is typical for the Autoencoders.

![image16.png](imgs/img16.png)

#### Build the Autoencoder model

In this example we'll use a basic fully-connected autoencoder but keep in mind that autoencoders can be built with different classes of neural network (i.e. Convolutional Neural Networks, Recurrent Neural Networks etc).

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2' # Avoid AVX2 error

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

def create_model(input_dim):
    # The encoder will consist of a number of dense layers that decrease in size
    # as we taper down towards the bottleneck of the network, the latent space
    input_data = Input(shape=(input_dim,), name='INPUT0')

    # hidden layers
    encoder = Dense(9, activation='tanh', name='encoder_1')(input_data)
    encoder = Dropout(.15)(encoder)
    encoder = Dense(6, activation='tanh', name='encoder_2')(encoder)
    encoder = Dropout(.15)(encoder)

    # bottleneck layer
    latent_encoding = Dense(3, activation='linear', name='latent_encoding')(encoder)

    # The decoder network is a mirror image of the encoder network
    decoder = Dense(6, activation='tanh', name='decoder_1')(latent_encoding)
    decoder = Dropout(.15)(decoder)
    decoder = Dense(9, activation='tanh', name='decoder_2')(decoder)
    decoder = Dropout(.15)(decoder)

    # The output is the same dimension as the input data we are reconstructing
    reconstructed_data = Dense(input_dim, activation='linear', name='OUTPUT0')(decoder)

    autoencoder_model = Model(input_data, reconstructed_data)

    return autoencoder_model

In [ ]:
autoencoder_model = create_model(len(features))
autoencoder_model.summary()

![image17.png](imgs/img17.png)

#### Model training

As we already explained, the autoencoder is a  type of artificial neural network used to learn efficient codings of unlabeled data. We'll use that to reconstruct the input at the output. To train an autoencoder we don’t need to do anything fancy, just throw the raw input data at it. Autoencoders are considered an unsupervised learning technique since they don’t need explicit labels to train on but to be more precise they are self-supervised because they generate their own labels from the training data.

To train our neural network we need to have a performance metric to measure how well it is learning to reconstruct the data i.e. our _loss function_. The loss function in our example, which we need to minimize during our training, is the error between the _input data_ and the _data reconstructed by the autoencoder_. We'll use the [Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error).

MSE = $\frac{1}{n}\sum_{i=1}^{n}{(Y_i - Y'_i)^2}$

Where:
- $n$: is the number of features (10 in our example)
- $Y_i$: is the original data point i.e. the input of the autoencoder 
- $Y'_i$: is the reconstructed data point i.e. the output of the autoencoder

Before starting the training we need to set the [**hyperparameters**](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)). Hyperparameters are parameters whose values control the learning process and determine the values of model parameters that a learning algorithm ends up learning. These are the `learning_rate`, `max_epochs`, `optimizer` and the `batch_size` you see in the code snippet below. You may ask yourself how to set them, it all comes down to trial and error. Try tweaking them below and see how they affect the learning process...

A good explaination of their meaning can be found in the [Keras documentation](https://keras.io/api/models/model_training_apis/).

In [ ]:
from tensorflow.keras import optimizers

batch_size = 32
max_epochs = 15
learning_rate = .0001

opt = optimizers.Adam(learning_rate=learning_rate)
autoencoder_model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
train_history = autoencoder_model.fit(x_train, x_train,
                      shuffle=True,
                      epochs=max_epochs,
                      batch_size=batch_size,
                      validation_data=(x_test, x_test))

In [ ]:
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.legend(['loss on train data', 'loss on test data'])

Here we can see the loss for the training set and the test set on the epochs.

Some of you might notice that this graph is somewhat unexpected. Why the validation loss is lower than the train loss? This is the effect of the regularization: regularization terms and dropout layer are affecting the network during training. A good writeup of this effect can be found [here](https://towardsdatascience.com/what-your-validation-loss-is-lower-than-your-training-loss-this-is-why-5e92e0b1747e).

As an excercise try and compute the average MSE on the training set and the test set. You'll find that the MSE is lower in the training set!

We can now save the model on disk as we'll use this later.

In [ ]:
autoencoder_model.save("./saved_model/autoencoder")

In [ ]:
!ls ./saved_model/autoencoder

---

### Model evaluation

We now have a model that reconstruct the input at the output... doesn't sounds really useful right?

Let's see it in action. Let's take a sample from the test set and run it through our autoencoder.

In [ ]:
input_sample = x_test[3:4].copy() # Deep copy

reconstructed_sample = autoencoder_model.predict(input_sample)

print(input_sample)
print(reconstructed_sample)

In [ ]:
import matplotlib.pyplot as plt

index = np.arange(10)
bar_width = 0.35

figure, ax = plt.subplots()

inbar = ax.bar(index, input_sample[0], bar_width, label="Input data")
recbar = ax.bar(index+bar_width, reconstructed_sample[0], bar_width, label="Reconstruced data")

ax.set_xlabel('Features')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(features, rotation = 45)
ax.legend()

As we can see from the graph above it reconstructed the input fairly well. It is not perfect since the Autoencoder is lossy but it is good enough

What happens if we manipulate this sample in a way the autoencoder doesn't expect (i.e. we introduce an **anomaly**)?

Let's try and set the `ACC_Z` to a value the autoencoder has never seen before.

In [ ]:
input_anomaly = input_sample.copy() # Deep copy

input_anomaly[0][2] = 0.15

reconstructed_anomaly = autoencoder_model.predict(input_anomaly)

print(input_anomaly)
print(reconstructed_anomaly)

In [ ]:
figure, ax = plt.subplots()

inbar = ax.bar(index, input_anomaly[0], bar_width, label="Input anomaly")
recbar = ax.bar(index+bar_width, reconstructed_anomaly[0], bar_width, label="Reconstruced anomaly")

ax.set_xlabel('Features')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(features, rotation = 45)
ax.legend()

The autoencoder fails to reconstruct the data it received at the input. This means that the reconstruction error is very high.

In [ ]:
from sklearn.metrics import mean_squared_error

print("Anomaly %f"% mean_squared_error(input_anomaly[0], reconstructed_anomaly[0]))
print("Normal  %f"% mean_squared_error(input_sample[0], reconstructed_sample[0]))

**It's working as expected!**

We now need to decide when to trigger an alarm (i.e. classify an input sample as anomalous) from this reconstruction error. In other words we need to decide our threshold.

There are multiple ways to set this value, in this example we'll use the [Z-Score](https://en.wikipedia.org/wiki/Standard_score).

From Wikipedia:
> In statistics, the standard score is the number of standard deviations by which the value of a raw score (i.e., an observed value or data point) is above or below the mean value of what is being observed or measured.[...]
>
> It is calculated by subtracting the population mean from an individual raw score and then dividing the difference by the population standard deviation.

We'll consider a sample an anomaly if the Reconstruction Error Z-Score is not in the range \[-2, +2\]. This means that if the reconstruction error for a sample is more than 2 standard deviation away from the average reconstruction error computed on the test set, the sample is an anomaly. This choice is arbirtary, we can control the sensitivity of the detector by changing this range.

In [ ]:
x_test_recon = autoencoder_model.predict(x_test)
reconstruction_scores = np.mean((x_test - x_test_recon)**2, axis=1)  # MSE

reconstruction_scores_pd = pd.DataFrame({'recon_score': reconstruction_scores})
print(reconstruction_scores_pd.describe())

In [ ]:
def z_score(mse_sample):
    return (mse_sample - reconstruction_scores_pd.mean())/reconstruction_scores_pd.std()

In [ ]:
mse_anomaly = mean_squared_error(input_anomaly[0], reconstructed_anomaly[0])
mse_normal = mean_squared_error(input_sample[0], reconstructed_sample[0])

z_score_anomaly = z_score(mse_anomaly)
z_score_normal = z_score(mse_normal)

print("Anomaly Z-score %f"% z_score_anomaly)
print("Normal Z-score %f"% z_score_normal)

We now have our anomaly detector... let's see how we can deploy it on our Kura&trade;-powered edge device.

---

## Model deployment

To deploy our model on the target device we'll leverage Kura&trade;'s newly added [Nvidia&trade; Triton Inferece Server](https://developer.nvidia.com/nvidia-triton-inference-server) integration.

![image18.png](imgs/img18.png)

The Nvidia™ Triton Inference Server is an open-source inference service software that enables the user to deploy trained AI models from any framework on GPU or CPU infrastructure. It supports all major frameworks like TensorFlow, TensorRT, PyTorch, ONNX Runtime, and even custom framework backend. With specific backends, it is also possible to run Python scripts, mainly for pre-and post-processing purposes, and exploit the DALI building block for optimized operations.

The Nvidia™ Triton Inference Server container should already be running on your system.

We'll also need to use Kura&trade;'s Triton bundles:
- [Triton Server Component](https://marketplace.eclipse.org/content/triton-server-component-eclipse-kura-5): for Kura-Triton integration
- [AI Wire Component](https://marketplace.eclipse.org/content/ai-wire-component-eclipse-kura-5): for making the Triton Inference Server available through the Kura Wires as a Wire component.

These bundles have been already installed on your Eclipse Kura&trade;

### Model conversion

The first step in using Triton to serve your models is to place one or more models into a [model repository](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_repository.md) i.e. a folder were the model are available for Triton to load. Depending on the type of the model and on what Triton capabilities you want to enable for the model, you may need to create a [model configuration](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md) for the model. This configuration is a protobuf containing informations about runtime configuration and input/output shape accepted by the model.

For our autoencoder model we'll need three "models":
- A **Preprocessor** for performing the operations described in the ["Data processing"](#Data-Processing) section (Wire envelop translation, feature selection and scaling)
- The **Autoencoder** model we exported in the ["Model training"](#Model-training) section
- A **Postprocessor** for performing the operations described in the ["Model evaluation"](#Model-evaluation) section (Reconstruction error computation)

To simplify the handling of these models and improve inference performance, we'll use an advanced feature of Triton wich is an [Ensemble Model](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/architecture.md#ensemble-models). From Triton official documentation:

> An ensemble model represents a pipeline of one or more models and the connection of input and output tensors between those models. Ensemble models are intended to be used to encapsulate a procedure that involves multiple models, such as "data preprocessing -> inference -> data postprocessing". Using ensemble models for this purpose can avoid the overhead of transferring intermediate tensors and minimize the number of requests that must be sent to Triton.

#### Autoencoder

As seen in the ["Model training"](#Model-training) section, our model is available as a [Tensorflow _SavedModel_](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_repository.md#tensorflow-models) which can be simply loaded by the Triton [Tensorflow backend](https://github.com/triton-inference-server/tensorflow_backend). We just need to configure it properly.

We'll start by creating the following folder structure

```
tf_autoencoder_fp32
├── 1
│   └── model.savedmodel
│       ├── assets
│       ├── keras_metadata.pb
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
└── config.pbtxt
```

This can be done by copying the model we saved in the Model Training section:

In [ ]:
!rm -rf ./tf_autoencoder_fp32/ && mkdir -p ./tf_autoencoder_fp32/1

In [ ]:
!ls

In [ ]:
cp -r ./saved_model/autoencoder tf_autoencoder_fp32/1/model.savedmodel

In [ ]:
!find tf_autoencoder_fp32 -maxdepth 3 -ls # or !tree tf_autoencoder_fp32

Now comes the hard part: we need to provide the [model configuration](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md) (i.e. the `config.pbtxt` file). In the case of the autoencoder is pretty simple:

```protobuf
name: "tf_autoencoder_fp32"
backend: "tensorflow"
max_batch_size: 0
input [
    {
    name: "INPUT0"
    data_type: TYPE_FP32
    dims: [ 1, 10 ]
    }
]
output [
    {
    name: "OUTPUT0"
    data_type: TYPE_FP32
    dims: [ -1, 10 ]
    }
]
version_policy: { all { }}
instance_group [{ kind: KIND_CPU }]
```

Each model `input` and `output` must specify the `name`, `data_type` and `dims`. We already know all of these:
- `name`: corresponds to the layer name we've seen in the Model Training section. `INPUT0` for the input and `OUTPUT0` for the output.
- `data_type`: will be float since we didn't perform any quantization
- `dims`: is the shape of the in/out tensor. In this case it will correspond to an array with the same length as the number of features.

Other interesting parameters of this configuration are:
- `backend`: where we set the backend for the model. In this case it will be the Tensorflow backend
- `name`: the name of the model that must correspond to the name of the folder
- `instance_group`: where we set where we want the model to run. In this case we'll use the CPU since we're on a Raspberry Pi but keep in mind that Triton support multiple accelerators.

for a deep dive into the model configuration parameter take a look at the [official documentation](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md).

#### Preprocessor

As discussed in the ["Data processing"](#Data-Processing) section, before providing the incoming data to the autoencoder, we need to perform feature selection and scaling. In addition to these responsibilites, the Preprocessor will need to perform a sort of serialization of the data to comply to the input shape accepted by the Autoencoder. This is due to how Kura manages the data running on Wires. More details can be found [here](https://eclipse.github.io/kura/docs-develop/kura-wires/single-port-wire-components/ai-wire-component/#models-input-and-output-formats).

To perform all of this we'll use the [Python backend](https://github.com/triton-inference-server/python_backend) available in Triton.

As described in the previous section we will need to provide the following folder structure:

```
preprocessor
├── 1
│   └── model.py
└── config.pbtxt
```

##### Preprocessor Configuration

As discussed in the [official Kura documentation](https://eclipse.github.io/kura/docs-develop/kura-wires/single-port-wire-components/ai-wire-component/#models-input-and-output-formats):

> The AI wire component takes a WireEnvelope as an input, it processes its records and feeds them to the specified preprocessing or inference model.
>
> ...
>
> The models that manage the input and the output must expect a list of inputs such that:
> - each input corresponds to an entry of the `WireRecord` properties
> - the entry key will become the input name (e.g. in the case of an asset, the channel name becomes the tensor name)
> - input shape will be `[1]`

Therefore for our `input` we'll have that each name corresponds to the names we've seen in the Data Collection section. The `output` needs to correspond to the input accepted by the model (i.e. `INPUT0`).

```protobuf
name: "preprocessor"
backend: "python"

input [
  {
    name: "ACC_X"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
input [
  {
    name: "ACC_Y"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
 ...
input [
  {
    name: "TEMP_PRESS"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
output [
  {
    name: "INPUT0"
    data_type: TYPE_FP32
    dims: [ 1, 10 ]
  }
]
instance_group [{ kind: KIND_CPU }]
```

##### Preprocessor Model

As we've seen in the Data Processing section the Preprocessor is responsible for scaling the input features and serializing them in the tensor shape expected by the Autoencoder model.

This can be done with the following python script:

```python
import numpy as np
import json

import triton_python_backend_utils as pb_utils


class TritonPythonModel:

    def initialize(self, args):
        self.model_config = model_config = json.loads(args['model_config'])

        output0_config = pb_utils.get_output_config_by_name(
            model_config, "INPUT0")

        self.output0_dtype = pb_utils.triton_string_to_numpy(
            output0_config['data_type'])

    def execute(self, requests):
        output0_dtype = self.output0_dtype

        responses = []

        for request in requests:
            acc_x      = pb_utils.get_input_tensor_by_name(request, "ACC_X").as_numpy()
            acc_y      = pb_utils.get_input_tensor_by_name(request, "ACC_Y").as_numpy()
            acc_z      = pb_utils.get_input_tensor_by_name(request, "ACC_Z").as_numpy()
            gyro_x     = pb_utils.get_input_tensor_by_name(request, "GYRO_X").as_numpy()
            gyro_y     = pb_utils.get_input_tensor_by_name(request, "GYRO_Y").as_numpy()
            gyro_z     = pb_utils.get_input_tensor_by_name(request, "GYRO_Z").as_numpy()
            humidity   = pb_utils.get_input_tensor_by_name(request, "HUMIDITY").as_numpy()
            pressure   = pb_utils.get_input_tensor_by_name(request, "PRESSURE").as_numpy()
            temp_hum   = pb_utils.get_input_tensor_by_name(request, "TEMP_HUM").as_numpy()
            temp_press = pb_utils.get_input_tensor_by_name(request, "TEMP_PRESS").as_numpy()

            out_0 = np.array([acc_y, acc_x, acc_z, pressure, temp_press, temp_hum, humidity, gyro_x, gyro_y, gyro_z]).transpose()

            #                  ACC_Y     ACC_X     ACC_Z    PRESSURE   TEMP_PRESS   TEMP_HUM   HUMIDITY    GYRO_X    GYRO_Y    GYRO_Z
            min = np.array([-0.132551, -0.049693, 0.759847, 976.001709, 38.724998, 40.220890, 13.003981, -1.937896, -0.265019, -0.250647])
            max = np.array([ 0.093099, 0.150289, 1.177543, 1007.996338, 46.093750, 48.355824, 23.506138, 1.923712, 0.219204, 0.671759])

            # MinMax scaling
            out_0_scaled = (out_0 - min)/(max - min)

            # Create output tensor
            out_tensor_0 = pb_utils.Tensor("INPUT0",
                                           out_0_scaled.astype(output0_dtype))

            inference_response = pb_utils.InferenceResponse(
                output_tensors=[out_tensor_0])
            responses.append(inference_response)

        return responses
```

Here there are two important things to note:
- The template we're using is taken from the Triton documentation and can be found [here](https://github.com/triton-inference-server/python_backend/blob/main/examples/add_sub/model.py).
- The MinMax scaling **must be the same we used in our training**. For illustration purposes we wrote the `min` and `max` arrays we found in the Data Processing section but we could have serialized the `MinMaxScaler` using [`pickle`](https://docs.python.org/3/library/pickle.html) instead.

#### Postprocessor

As discussed in the ["Data processing"](#Data-Processing) section, to perform the anomaly detection step we need to compute the Mean Squared Error between the recontructed data and the actual input data. Due to this the configuration of the Postprocessor model will be somewhat more complicated than before: in addition to the output of the Autoencoder model we will need the output of the Preprocessor model.

To perform all of this we'll use the [Python backend](https://github.com/triton-inference-server/python_backend) again.

As described in the previous section we will need to provide the following folder structure:

```
postprocessor
├── 1
│   └── model.py
└── config.pbtxt
```

##### Postprocessor Configuration

```protobuf
name: "postprocessor"
backend: "python"

input [
  {
    name: "RECONSTR0"
    data_type: TYPE_FP32
    dims: [ 1, 10 ]
  }
]
input [
  {
    name: "ORIG0"
    data_type: TYPE_FP32
    dims: [ 1, 10 ]
  }
]
output [
  {
    name: "ANOMALY_SCORE0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
output [
  {
    name: "ANOMALY0"
    data_type: TYPE_BOOL
    dims: [ 1 ]
  }
]
instance_group [{ kind: KIND_CPU }]
```

As we can see we have two inputs and two outputs:
- The first input tensor is the reconstruction performed by the autoencoder model
- The second input tensor is the original data (already scaled and serialized by the Preprocessor model)
- The first output is the anomaly score i.e. the reconstruction error between the original and the reconstructed data.
- The second output is a boolean representing whether the data constitute an anomaly or not

Let's see how this is computed by the Python model.

##### Postprocessor Model

```python
import numpy as np
import json

import triton_python_backend_utils as pb_utils

def z_score(mse):
    return (mse - MEAN_MSE)/STD_MSE


class TritonPythonModel:

    def initialize(self, args):
        self.model_config = model_config = json.loads(args['model_config'])

        output0_config = pb_utils.get_output_config_by_name(
            model_config, "ANOMALY_SCORE0")
        output1_config = pb_utils.get_output_config_by_name(
            model_config, "ANOMALY0")

        self.output0_dtype = pb_utils.triton_string_to_numpy(
            output0_config['data_type'])
        self.output1_dtype = pb_utils.triton_string_to_numpy(
            output1_config['data_type'])

    def execute(self, requests):
        output0_dtype = self.output0_dtype
        output1_dtype = self.output1_dtype

        responses = []

        for request in requests:
            # Get input
            x_recon = pb_utils.get_input_tensor_by_name(request, "RECONSTR0").as_numpy()
            x_orig = pb_utils.get_input_tensor_by_name(request, "ORIG0").as_numpy()

            # Get Mean square error between reconstructed input and original input
            reconstruction_score = np.mean((x_orig - x_recon)**2, axis=1)
            
            # Z-Score of Mean square error must be inside [-2; 2]
            anomaly = np.array([z_score(reconstruction_score) < -2.0 or z_score(reconstruction_score) > 2.0])

            # Create output tensors
            out_tensor_0 = pb_utils.Tensor("ANOMALY_SCORE0",
                                           reconstruction_score.astype(output0_dtype))
            out_tensor_1 = pb_utils.Tensor("ANOMALY0",
                                           anomaly.astype(output1_dtype))

            inference_response = pb_utils.InferenceResponse(
                output_tensors=[out_tensor_0, out_tensor_1])
            responses.append(inference_response)

        return responses
```

As you can see the script is simple:
- It gets the input tensors
- It computes the Mean Squared Error between the inputs (which is what we called the reconstruction error)
- It computes the Z-Score of the MSE computed for the current sample and flags it as an anomaly if it is farther than 2 standard deviations away from the average MSE.

**Note**: `MEAN_MSE` and `STD_MSE` are the mean value and the standard deviation of the Mean Squared Error computed on the test set and correspond to the `reconstruction_scores_pd.mean()` and `reconstruction_scores_pd.std()` we used in the previous section. We didn't set them as they change for every training performed on the Autoencoder. Be sure to set it to their proper values before trying this model on the Triton server!

### Ensemble model

To make things easier for ourselves and improve performance we'll consolidate the AI pipeline into an [Ensemble Model](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/architecture.md#ensemble-models).

We will need to provide the following folder structure:

```
ensemble_pipeline
├── 1
└── config.pbtxt
```

Note that the `1` folder is **empty**. The ensemble model essentially describe *how to connect the models that belong to the processing pipeline*.

Therefore we'll need to focus on the configuration only.

```protobuf
name: "ensemble_pipeline"
platform: "ensemble"
max_batch_size: 0
input [
  {
    name: "ACC_X"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
input [
  {
    name: "ACC_Y"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
 ...
input [
  {
    name: "TEMP_PRESS"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
output [
  {
    name: "ANOMALY_SCORE0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
]
output [
  {
    name: "ANOMALY0"
    data_type: TYPE_BOOL
    dims: [ 1 ]
  }
]
ensemble_scheduling {
  step [
    {
      model_name: "preprocessor"
      model_version: -1
      input_map{
          key: "ACC_X"
          value: "ACC_X"
      }
      input_map{
          key: "ACC_Y"
          value: "ACC_Y"
      }
       ...
      input_map{
          key: "TEMP_PRESS"
          value: "TEMP_PRESS"
      }
      output_map {
        key: "INPUT0"
        value: "preprocess_out"
      }
    },
    {
      model_name: "tf_autoencoder_fp32"
      model_version: -1
      input_map {
        key: "INPUT0"
        value: "preprocess_out"
      }
      output_map {
        key: "OUTPUT0"
        value: "autoencoder_output"
      }
    },
    {
      model_name: "postprocessor"
      model_version: -1
      input_map {
        key: "RECONSTR0"
        value: "autoencoder_output"
      }
      input_map {
        key: "ORIG0"
        value: "preprocess_out"
      }
      output_map {
        key: "ANOMALY_SCORE0"
        value: "ANOMALY_SCORE0"
      }
      output_map {
        key: "ANOMALY0"
        value: "ANOMALY0"
      }
    }
  ]
}
```

The configuration is split in two main parts:
- The first is the usual configuration we've seen before: we describe what are the input and the output of our model. In this case the input will correspond to the input of the first model of the pipeline (the Preprocessor) and the output to the output of the last model of the pipeline (the Postprocessor)
- The second part describe how to map the input/output of the models within the pipeline

To better visualize the configuration we can look at the graph below.

![image19.png](imgs/img19.png)

#### Conversion results

At this point we should have a folder structure that looks like this:

```
models
├── ensemble_pipeline
│   ├── 1
│   └── config.pbtxt
├── postprocessor
│   ├── 1
│   │   └── model.py
│   └── config.pbtxt
├── preprocessor
│   ├── 1
│   │   └── model.py
│   └── config.pbtxt
└── tf_autoencoder_fp32
    ├── 1
    │   └── model.savedmodel
    │       ├── assets
    │       ├── keras_metadata.pb
    │       ├── saved_model.pb
    │       └── variables
    │           ├── variables.data-00000-of-00001
    │           └── variables.index
    └── config.pbtxt
```

Now let's go back to our tutorial file !